In [1]:
import pandas as pd

In [57]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [3]:
gln_path = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S2to5combined_20ul_40min.xlsx'
df_mock = load_data(gln_path)
gln_path = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S6to10combined_20ul_40min.xlsx'
df_rsv = load_data(gln_path)

In [95]:
fpath = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/UTMB1.xlsx'
df_intact_mock = load_data(fpath)
fpath = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/UTMB6.xlsx'
df_intact_rsv = load_data(fpath)

df_intact_mock = df_intact_mock[(df_intact_mock.Mass>24000)&(df_intact_mock.Mass<28000)]
df_intact_rsv = df_intact_rsv[df_intact_rsv.Mass>24000]

In [96]:
_, df_only_intact_rsv = diff_dfs(df_intact_mock, df_intact_rsv)

In [9]:
df_mock = df_mock[df_mock.Mass>1500]
df_rsv = df_rsv[df_rsv.Mass>1500]

In [97]:
plotly_zone(df_intact_mock)
plotly_zone(df_intact_rsv)

In [23]:
df_mock.shape, df_rsv.shape

((3285, 5), (1469, 5))

In [29]:
dfm_comm_rsv = match_dfs(df_mock, df_rsv)
dfm_comm_mock = match_dfs(df_rsv, df_mock)
dfm_only_rsv = df_rsv.drop(dfm_comm_rsv.index)
dfm_only_mock = df_mock.drop(dfm_comm_mock.index)

In [51]:
df_mock_top = local_top(df_mock)
df_rsv_top = local_top(df_rsv)
df_mock_top.shape, df_rsv_top.shape

((271, 5), (239, 5))

In [58]:
_, df_rsv_top_only = diff_dfs(df_mock_top, df_rsv_top)

In [106]:
# remove adducts from df_gln2
adducts = [K, Na]
dfm_adducts_list = list()
for adduct in adducts:
    dfm_r = match_dfs(df_mock, df_mock, shift=adduct)
    dfm_adducts_list.append(dfm_r)
dfm_adducts = pd.concat(dfm_adducts_list).drop_duplicates()
df_mock_without_adducts = df_mock.drop(dfm_adducts.index)

In [107]:
# remove adducts from df_gln2
adducts = [K, Na]
dfm_adducts_list = list()
for adduct in adducts:
    dfm_r = match_dfs(df_rsv, df_rsv, shift=adduct)
    dfm_adducts_list.append(dfm_r)
dfm_adducts = pd.concat(dfm_adducts_list).drop_duplicates()
df_rsv_without_adducts = df_rsv.drop(dfm_adducts.index)

In [108]:
plotly_zone(df_mock)
plotly_zone(df_rsv)

In [109]:
import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

def func(df_s1, df_s2, shift):
    dfm = match_dfs(df_s1, df_s2, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

PROCESSES = 16
params = [(df_mock, df_rsv, shift) for shift in np.arange(-64, 64, 1)]
# params = [(df_intact_mock, df_only_intact_rsv, shift) for shift in np.arange(-64, 64, 1)]
# params = [(df_mock, df_rsv, shift) for shift in np.arange(1, 300, 1)]
# params = [(df_mock, df_rsv, shift) for shift in np.arange(1, 300, 1)]
# params = [(df_mock_top, df_rsv_top_only, shift) for shift in np.arange(-64, 64, 1)]
# params = [(df_mock, dfm_only_rsv, shift) for shift in np.arange(-64, 64, 1)]
# params = [(df_sample, shift) for shift in np.arange(1, 64, 0.1)]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits = pool.starmap(func, params)

df_shift_hits = pd.DataFrame(shift_hits)    

In [111]:
fig = px.scatter(df_shift_hits, x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'})
fig.show()

In [112]:
df_shift_hits.sort_values('hit', ascending=False).reset_index(drop=True).iloc[:10].copy()

,shift,hit
0,0,738
1,-1,424
2,1,408
3,-15,389
4,15,378
5,-16,377
6,-31,363
7,14,357
8,-14,356
9,-40,355


In [89]:
# df_sh_mock2rsv = df_shift_hits.copy()
df_sh_intact_mock2rsv = df_shift_hits.copy()

In [68]:
shift = -1

# dfm_r = match_dfs(df_mock, dfm_only_rsv, shift=shift)
# dfm_l = match_dfs(dfm_only_rsv, df_mock, shift=-1*shift)

dfm_mock = match_dfs(df_intact_mock, df_only_intact_rsv, shift=shift)
dfm_rsv = match_dfs(df_rsv, df_rsv, shift=shift)

# dfm.shape, dfm_mock.shape, dfm_rsv.shape

In [113]:
shifts = [-16, -15, -1, 1, 15]
dfm_list = list()
for shift in shifts:
    dfm_r = match_dfs(df_mock, df_rsv, shift=shift)
    dfm_list.append(dfm_r)
# dfm_list.append(df_mock)
dfm_shifts = pd.concat(dfm_list).drop_duplicates()

In [115]:
# plotly_zones(dfm_l, dfm_r)
# plotly_zone(dfm_mock)
# plotly_zone(dfm_rsv)
plotly_zone(dfm_shifts)

In [70]:
dfm_l.shape

(160, 5)

In [71]:
bcr = base_calling_random(dfm_l)
plotly_basecalling(*bcr)

mass_x (160,) mass_y (160,)
